<a href="https://colab.research.google.com/github/hiwei93/llm-practice/blob/main/hf_NLP_course_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Huggingface NLP Course: Fine-tuning

https://huggingface.co/learn/nlp-course/chapter3/2?fw=pt

In [ ]:
!pip install 'transformers[torch]' datasets accelerate evaluate -U -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━

## 数据准备

### 查看数据情况

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [ ]:
raw_train_dataset = raw_datasets['train']
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [ ]:
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [ ]:
raw_train_dataset[15]

{'sentence1': 'Rudder was most recently senior vice president for the Developer & Platform Evangelism Business .',
 'sentence2': 'Senior Vice President Eric Rudder , formerly head of the Developer and Platform Evangelism unit , will lead the new entity .',
 'label': 0,
 'idx': 16}

### tokenize 文本

1. 多个文本同时 tokenize：
    - padding 自动填充，返回数据中有attention_mask字段，用于表名哪些id是填充的，不会用于训练
2. 数据集批量操作：批量 tokenize
   - dataset.map 方法
3. 动态填充 dynamic padding，样本批处理
   - 将一组数据的id进行填充，保证张量长度相同
   - DataCollatorWithPadding

In [ ]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
tokenizer.pad_token_id

0

In [ ]:
tokens = tokenizer.tokenize("This is the first sentence.")
tokens

['this', 'is', 'the', 'first', 'sentence', '.']

In [ ]:
ids = tokenizer.convert_tokens_to_ids(tokens)
ids

[2023, 2003, 1996, 2034, 6251, 1012]

In [ ]:
tokenizer.decode(ids)

'this is the first sentence.'

In [ ]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")

In [ ]:
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['[CLS]',
 'this',
 'is',
 'the',
 'first',
 'sentence',
 '.',
 '[SEP]',
 'this',
 'is',
 'the',
 'second',
 'one',
 '.',
 '[SEP]']

In [ ]:
tokenized_dataset = tokenizer(
    raw_train_dataset['sentence1'],
    raw_train_dataset['sentence2'],
    padding=True,
    truncation=True
)

In [ ]:
tokenized_dataset.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
samples = tokenized_datasets['train'][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32]

In [ ]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 67]),
 'token_type_ids': torch.Size([8, 67]),
 'attention_mask': torch.Size([8, 67]),
 'labels': torch.Size([8])}

### 练习

raw_datasets = load_dataset("glue", "sst2")

In [ ]:
from datasets import load_dataset
raw_datasets = load_dataset("glue", "sst2")

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [ ]:
raw_datasets['train'][0]

{'sentence': 'hide new secretions from the parental units ',
 'label': 0,
 'idx': 0}

In [ ]:
raw_datasets['train'].features

{'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'positive'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [ ]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenize_function(example):
    return tokenizer(example["sentence"], truncation=True)

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
samples = tokenized_datasets['train'][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence"]}

In [ ]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 29]),
 'token_type_ids': torch.Size([8, 29]),
 'attention_mask': torch.Size([8, 29]),
 'labels': torch.Size([8])}

### 查看ChatGLM3 tokenizer

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm3-6b", trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

tokenization_chatglm.py:   0%|          | 0.00/13.0k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm3-6b:
- tokenization_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer.model:   0%|          | 0.00/1.02M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]

In [ ]:
tokens = tokenizer.tokenize("我爱北京天安门，天安门上太阳升。")
tokens

['▁我', '爱', '北京', '天', '安门', '，', '天', '安门', '上', '太阳', '升', '。']

In [ ]:
ids = tokenizer.convert_tokens_to_ids(tokens)
ids

[34211, 54799, 31719, 54614, 48277, 31123, 54614, 48277, 54547, 33146, 54990]

In [ ]:
inputs = tokenizer("我爱北京天安门，", "天安门上太阳升。")
inputs

{'input_ids': [64790, 64792, 34211, 54799, 31719, 54614, 48277, 31123, 36718, 48277, 54547, 33146, 54990, 31155, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'position_ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]}

In [ ]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['[gMASK]',
 'sop',
 '▁我',
 '爱',
 '北京',
 '天',
 '安门',
 '，',
 '▁天',
 '安门',
 '上',
 '太阳',
 '升',
 '。',
 '']

## 微调模型

In [ ]:
# 数据准备
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

### 配置超参数

这些参数用于训练和评估

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    "bert-finetuned-mrpc",
)

针对特定任务，加载对应的模型类

不同的任务都会有不同的 head，将结果进行映射， bert 基础模型没有经过分类任务的训练，所以会丢去原有的head，增加分类的head。新增的权重需要进行训练，从而适应新的任务。

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.618000
2,0.616800,0.555490
3,0.487200,0.489061


TrainOutput(global_step=1377, training_loss=0.49676589023919754, metrics={'train_runtime': 204.7023, 'train_samples_per_second': 53.756, 'train_steps_per_second': 6.727, 'total_flos': 405114969714960.0, 'train_loss': 0.49676589023919754, 'epoch': 3.0})

### 评价

构建评价指标

In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


In [ ]:
print(predictions.predictions[0][1], predictions.label_ids[0])

2.3586025 1


In [ ]:
import numpy as np

# 将 logits 转换成 label
preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
preds[0]

1

In [ ]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.3161764705882353, 'f1': 0.0}

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

### 带有评价指标的 trainer

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    "bert-finetuned-mrpc",
    evaluation_strategy="epoch",
)

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  # 计算性能指标
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.352807,0.860294,0.900175
2,0.534900,0.476234,0.865196,0.905983
3,0.299400,0.679362,0.862745,0.905724


TrainOutput(global_step=1377, training_loss=0.34571976339635246, metrics={'train_runtime': 197.7835, 'train_samples_per_second': 55.637, 'train_steps_per_second': 6.962, 'total_flos': 405114969714960.0, 'train_loss': 0.34571976339635246, 'epoch': 3.0})

### 发布微调模型到 Huggingface

In [ ]:
from google.colab import userdata

training_args.hub_token = userdata.get('HF_TOKEN')
trainer.push_to_hub(model_name="bert-finetuned-mrpc")

events.out.tfevents.1711295193.46fd881f037d.1522.0:   0%|          | 0.00/6.51k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/hiwei/test-trainer/commit/ee9b5a214986d67a67993eb7ced55789ba91ec3a', commit_message='End of training', commit_description='', oid='ee9b5a214986d67a67993eb7ced55789ba91ec3a', pr_url=None, pr_revision=None, pr_num=None)

### 使用微调模型

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_id = "hiwei/bert-finetuned-mrpc"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)

In [ ]:
inputs = tokenizer("I like dogs", "I am a dog person", return_tensors="pt")
predictions = model(**inputs)
preds = np.argmax(predictions.logits.detach().numpy(), axis=-1)
preds

array([1])

In [ ]:
# Pipeline 用法
from transformers import pipeline

pipe = pipeline("text-classification", model="hiwei/bert-finetuned-mrpc")
pipe({"text": "I like dogs", "text_pair": "I love my dog"})

{'label': 'LABEL_1', 'score': 0.9889406561851501}

### 练习

前提：确认训练任务的类型

1. 获取数据
2. tokenize 数据并批量处理
3. 配置模型微调参数
4. 创建模型性能评估方法
5. 实例化 trainer 实例
6. 开始训练
7. 上传模型
8. 使用模型完成任务

#### 加载数据

In [ ]:
# 数据准备
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "sst2")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

#### 性能评估指标

In [ ]:
import evaluate
import numpy as np

def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "sst2")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

#### 配置并实例化 Trainer

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    "bert-finetuned-sst2",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    push_to_hub=True
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  # 计算性能指标
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.279900,0.406847,0.888761
2,0.205400,0.411675,0.899083
3,0.114600,0.412962,0.907110


TrainOutput(global_step=25257, training_loss=0.2255641445485091, metrics={'train_runtime': 2177.1681, 'train_samples_per_second': 92.803, 'train_steps_per_second': 11.601, 'total_flos': 3082513027395900.0, 'train_loss': 0.2255641445485091, 'epoch': 3.0})

#### 发布到 huggingface

In [ ]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/hiwei/bert-finetuned-sst2/commit/cf0a718ccf9e0069ae5c46abad6e4a4be50b7348', commit_message='End of training', commit_description='', oid='cf0a718ccf9e0069ae5c46abad6e4a4be50b7348', pr_url=None, pr_revision=None, pr_num=None)

#### 使用微调模型

In [ ]:
# Pipeline 用法
from transformers import pipeline

pipe = pipeline("text-classification", model="hiwei/bert-finetuned-sst2")
pipe("I like dogs")

## 帮助方法

In [ ]:
del trainer

In [ ]:
import torch
torch.cuda.empty_cache()